In [ ]:
# =============================================================================
# 📘 1. 라이브러리 임포트 및 설정
# =============================================================================
import firebase_admin
from firebase_admin import credentials, firestore
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree, _tree
from sklearn.metrics import classification_report, confusion_matrix

# ⚠️ [중요] 본인의 키 파일 경로로 수정하세요! (역슬래시 \ 대신 슬래시 / 사용 권장)
YOUR_KEY_PATH = r"C:\Users\USER\.jupyter\lab\workspaces\capstone-sleep-firebase-adminsdk-fbsvc-f875e5fa9b.json"
# Firebase 초기화 (이미 되어있으면 패스)
if not firebase_admin._apps:
    cred = credentials.Certificate(YOUR_KEY_PATH)
    firebase_admin.initialize_app(cred)
db = firestore.client()
print("✅ Firebase 연결 성공!")

# =============================================================================
# 📗 2. 데이터 가져오기 (Raw Data Load)
# =============================================================================
print("📥 데이터 다운로드 중... (시간이 좀 걸릴 수 있습니다)")
# 라벨이 있는(훈련용) 데이터만 가져옵니다.
docs = db.collection('raw_data').where('label', '!=', None).stream()

data_list = []
for doc in docs:
    d = doc.to_dict()
    data_list.append({
        'hr': d.get('hr', 0),
        'spo2': d.get('spo2', 0),
        # 최신 이름(avg)을 우선하되, 혹시 모를 옛날 데이터(level)도 호환
        'mic_avg': d.get('mic_avg', d.get('mic_level', 0)), 
        'pressure_avg': d.get('pressure_avg', d.get('pressure_level', 0)),
        'label': d.get('label')
    })

df = pd.DataFrame(data_list)

# 데이터 정제 (0인 값이나 이상한 값 제거)
df = df[df['label'] != 'Unknown'] 
df = df.dropna()

print(f"✅ 총 {len(df)}개의 고품질 훈련 데이터 로드 완료!")
print("\n📊 라벨별 데이터 개수:")
print(df['label'].value_counts())

# =============================================================================
# 📙 3. AI 모델 학습 (Decision Tree)
# =============================================================================
# 학습에 사용할 특성(X)과 정답(y) 정의
feature_names = ['hr', 'spo2', 'mic_avg', 'pressure_avg']
X = df[feature_names]
y = df['label']

# 훈련셋/테스트셋 분리 (80% 학습, 20% 검증)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 생성 및 학습 (너무 복잡하지 않게 깊이 제한)
clf = DecisionTreeClassifier(max_depth=5, random_state=42)
clf.fit(X_train, y_train)

accuracy = clf.score(X_test, y_test) * 100
print(f"\n🚀 AI 모델 학습 완료! 정확도: {accuracy:.2f}%")

# =============================================================================
# 🎨 4. [발표용] 시각화 자료 생성 (스크린샷 찍으세요!)
# =============================================================================

# 1) 데이터 분포도 (Pairplot) - 데이터가 뭉쳐있는 특징을 보여줌
print("\n📊 [발표 자료 1] 데이터 분포도 생성 중...")
sns.set(style="ticks")
sns.pairplot(df, hue="label", palette="bright", vars=feature_names)
plt.suptitle(f"Sensor Data Distribution (Accuracy: {accuracy:.1f}%)", y=1.02)
plt.show()

# 2) 혼동 행렬 (Confusion Matrix) - AI가 얼마나 잘 맞췄는지 보여줌
print("\n📊 [발표 자료 2] 예측 정확도 히트맵 생성 중...")
y_pred = clf.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=clf.classes_, yticklabels=clf.classes_)
plt.xlabel('AI Prediction')
plt.ylabel('Actual Label')
plt.title('AI Model Accuracy (Confusion Matrix)')
plt.show()

# 3) 의사결정 나무 (Decision Tree) - AI의 판단 로직 시각화
print("\n📊 [발표 자료 3] 의사결정 나무 시각화...")
plt.figure(figsize=(25, 10))
plot_tree(clf, feature_names=feature_names, class_names=clf.classes_, filled=True, rounded=True, fontsize=10)
plt.title('Sleep Stage Decision Tree Logic')
plt.show()

# =============================================================================
# 💾 5. [핵심] Python 함수 코드 추출 (main.py에 붙여넣을 것)
# =============================================================================
def tree_to_code(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    
    code_lines = []
    code_lines.append("def predict_stage_by_tree(hr: float, spo2: float, mic_avg: float, pressure_avg: float) -> str:")
    code_lines.append(f"    # AI Model Accuracy: {accuracy:.2f}%")
    code_lines.append(f"    # Trained on {len(df)} samples")
    
    def recurse(node, depth):
        indent = "    " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            code_lines.append(f"{indent}if {name} <= {threshold:.4f}:")
            recurse(tree_.children_left[node], depth + 1)
            code_lines.append(f"{indent}else:  # {name} > {threshold:.4f}")
            recurse(tree_.children_right[node], depth + 1)
        else:
            # 가장 확률 높은 클래스 선택
            class_idx = np.argmax(tree_.value[node][0])
            class_name = class_names[class_idx]
            code_lines.append(f"{indent}return \"{class_name}\"")

    recurse(0, 1)
    return "\n".join(code_lines)

print("\n" + "="*50)
print("👇 아래 코드를 복사해서 functions/main.py 에 덮어쓰세요! 👇")
print("="*50)
print(tree_to_code(clf, feature_names, list(clf.classes_)))
print("="*50)

TypeError: Metaclasses with custom tp_new are not supported.